## Annotated CycleGAN

As we are building a web app "deepfake" which is of image translation based on GAN model, I present an "annotated" version of the pytorch-based-CycleGAN model in the form of a line-by-line implementation. It will cover data preparation, different GAN models, training schemes, optimization tricks for both training and computation. After completing this notebook, you will know:

- How to implement the discriminator and generator models.
- How to define composite models to train the two aforementioned models via adversarial and cycle loss.
- How to do the model inference to generate the translated image.

To follow along you will first need to install the libraries listed in `requirements-an.txt`. The workflow should be considered as a starting point of image translation tasks with the enrichment of GAN model.

> My comments are blockquoted. The main text is based on the CycleGAN/Pytorch documentation.
> Wangsu Hu (wangsu.hu1234@gmail.com)

## Prelims

In [4]:
!pip install -q -r requirements-an.txt

You should consider upgrading via the '/Users/wangsuhu/Google Drive/Github/pytorch-CycleGAN-and-pix2pix/env/bin/python3 -m pip install --upgrade pip' command.


## Background

The Cycle Generative Adversarial Network, or CycleGAN, is an approach to training a deep convolutional neural network for image-to-image translation tasks.

Unlike other GAN models for image translation, the CycleGAN does not require a dataset of paired images. 

The CycleGAN model was described by Jun-Yan Zhu, et al. in their 2017 paper titled [“Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks.”](https://arxiv.org/abs/1703.10593)

Here is the official github repo released and maintained by the author list [pytorch-CycleGAN-and-pix2pix](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix). 

The model architecture is comprised of two generator models: 
- one generator (Generator G_A2B) for generating images for the second class (class-B) based on the input image for the first class (class-A)
- another generator (Generator G_B2A) for generating images for the first class (class-A) based on the input image for the second class (class-B)

> as you can see, unlike normal GAN using randomness, the generator of CycleGAN is conditional on the input image and the input image can be a real image or a generated image. See the generator blocks in workflow below.

-- real image for class-A -> G_A2B -> generated image for class-B 
-- generated image for class-A -> G_A2B -> generated image for class-B 
-- real image for class-B -> G_B2A -> generated image for class-A 
-- generated image for class-B -> G_B2A -> generated image for class-A 

- one discriminator (Discriminator D_A) takes images from class-A then predicts whether they are real or fake.
- one discriminator (Discriminator D_B) takes images from class-B then predicts whether they are real or fake.

> similar to generators, the input image of discriminator can be a real image or a generated image.

-- real image for class-A -> D_A -> real/fake
-- generated image for class-A -> D_A -> real/fake
-- real image for class-B -> D_B -> real/fake
-- generated image for class-B -> D_B -> real/fake


Why does sucg GAN model called "Cycle"? 

The discriminator and generator models are trained in an adversarial zero-sum process, like normal GAN models. The generators learn to better fool the discriminators and the discriminator learn to better detect fake images. Together, the models find an equilibrium during the training process.

Additionally, the generator models are regularized to not just create new images in the target domain, but instead translate more reconstructed versions of the input images from the source domain. This is achieved by using generated images as input to the corresponding generator model and comparing the output image to the original images. Passing an image through both generators is called a cycle. Together, each pair of generator models are trained to better reproduce the original source image, referred to as cycle consistency.

-- real image for class-A -> G_A2B -> generated image for class-B -> G_B2A -> generated image for class-A vs the input real image for class-A
-- real image for class-B -> G_B2A -> generated image for class-A -> G_A2B -> generated image for class-B vs the input real image for class-B

see the following flow chart for your reference.
![work_flow](workflow.png)


> we dont consider the "identity mapping" architecture here for simplicity.

## Prepare Data